In [1]:
# !pip install opencv-python matplotlib tensorflow-datasets

In [42]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import uuid

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Conv2D, MaxPooling2D, Input, Flatten
import tensorflow as tf
import tensorflow_datasets as tfds

2024-08-31 12:26:44.332191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-31 12:26:44.349681: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-31 12:26:44.354577: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-31 12:26:44.367096: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-31 12:26:45.234366: W tensorflow/compiler/tf2

# Set GPU Growth

In [4]:
#avoid out of memory erros by setting GPU memory growth

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


I0000 00:00:1725118005.865099    8639 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725118005.916465    8639 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725118005.916898    8639 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [5]:
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


## Folder structures

In [6]:
POS_PATH = os.path.join("data", "positive")
NEG_PATH = os.path.join("data", "negative")
ANC_PATH = os.path.join("data", "anchor")

In [7]:
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

In [ ]:
lfw_dataset, info = tfds.load('lfw', split='train', with_info=True, as_supervised=True)

# Display dataset information
print(info)

In [36]:
# Function to save images to a directory
def save_image(image, index, directory):
    # Convert the image tensor to a NumPy array and then to uint8
    image_np = tf.image.convert_image_dtype(image, dtype=tf.uint8).numpy()

    # Encode image in JPEG format
    image_encoded = tf.image.encode_jpeg(image_np)
    
    # Define the path to save the image
    image_path = os.path.join(directory, f'image_{index}.jpg')
    
    # Write the image to the file
    tf.io.write_file(image_path, image_encoded)

# Move all images to the negative directory
for index, (label, image) in enumerate(lfw_dataset):
    save_image(image, index, NEG_PATH)

print(f"All images have been moved to the {NEG_PATH} directory.")

All images have been moved to the data/negative directory.


## Colect Positive and Anchor Classes

In [38]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

    if cv2.waitKey(1) & 0XFF == ord('p'):
        cv2.imwrite(os.path.join(POS_PATH, f"image_{random.randint(0, 1000)}.jpg"), frame)  
    
cap.release()
cv2.destroyAllWindows()

In [44]:
#Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        (x, y, w, h) = faces[0] 
        center_x, center_y = x + w // 2, y + h // 2
        
        # Define the dimensions for the centered crop
        half_width = 125  # Half of the target width 250
        half_height = 125  # Half of the target height 250

        
        start_x = max(center_x - half_width, 0)
        start_y = max(center_y - half_height, 0)
        end_x = min(center_x + half_width, frame.shape[1])
        end_y = min(center_y + half_height, frame.shape[0])
        
        
        cropped_frame = frame[start_y:end_y, start_x:end_x]
        
        
        resized_frame = cv2.resize(cropped_frame, (250, 250))
    else:
        
        resized_frame = cv2.resize(frame, (250, 250))
    
    
    cv2.imshow('frame', resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('p'):
        image_path = os.path.join(POS_PATH, f"{uuid.uuid1()}.jpg")
        cv2.imwrite(image_path, resized_frame)
        print(f"Saved frame to {image_path}")

    if cv2.waitKey(1) & 0xFF == ord('a'):
        image_path = os.path.join(ANC_PATH, f"{uuid.uuid1()}.jpg")
        cv2.imwrite(image_path, resized_frame)
        print(f"Saved frame to {image_path}")
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    

cap.release()
cv2.destroyAllWindows()

Saved frame to data/anchor/e31c760e-67be-11ef-8b57-dc215cb50b44.jpg
Saved frame to data/anchor/e68fe1a4-67be-11ef-8b57-dc215cb50b44.jpg
Saved frame to data/anchor/e764c8a6-67be-11ef-8b57-dc215cb50b44.jpg
Saved frame to data/anchor/e7c43502-67be-11ef-8b57-dc215cb50b44.jpg
Saved frame to data/anchor/ee1cf5a6-67be-11ef-8b57-dc215cb50b44.jpg
Saved frame to data/anchor/ee36e0ce-67be-11ef-8b57-dc215cb50b44.jpg
Saved frame to data/anchor/eed7fed2-67be-11ef-8b57-dc215cb50b44.jpg
Saved frame to data/anchor/eef326e4-67be-11ef-8b57-dc215cb50b44.jpg
